In [1]:
import pandas as pd

df = pd.read_csv('../data/cubes_processed.csv')
df

,Cube ID,Timestamp,Temperature Layer A,Temperature Layer B,Humidity Layer A,Humidity Layer B
0,83,0,23.031810,22.393901,9.915392,8.932601
1,87,0,23.251756,23.859351,9.542380,8.506288
2,50,8,25.005865,23.974537,9.861271,9.919887
3,40,21,25.315756,24.065631,8.915186,10.003067
4,2,21,22.809779,23.765559,10.086120,8.637221
...,...,...,...,...,...,...
239209,81,4227968,19.840444,20.517986,8.983224,8.146869
239210,70,4227984,19.914762,20.214499,10.171405,8.755614
239211,34,4227987,19.122549,20.760434,9.814049,7.426978
239212,17,4228183,21.117863,21.011024,8.622131,9.234980


In [2]:
import numpy as np

df['Target'] = np.nan
df

,Cube ID,Timestamp,Temperature Layer A,Temperature Layer B,Humidity Layer A,Humidity Layer B,Target
0,83,0,23.031810,22.393901,9.915392,8.932601,NaN
1,87,0,23.251756,23.859351,9.542380,8.506288,NaN
2,50,8,25.005865,23.974537,9.861271,9.919887,NaN
3,40,21,25.315756,24.065631,8.915186,10.003067,NaN
4,2,21,22.809779,23.765559,10.086120,8.637221,NaN
...,...,...,...,...,...,...,...
239209,81,4227968,19.840444,20.517986,8.983224,8.146869,NaN
239210,70,4227984,19.914762,20.214499,10.171405,8.755614,NaN
239211,34,4227987,19.122549,20.760434,9.814049,7.426978,NaN
239212,17,4228183,21.117863,21.011024,8.622131,9.234980,NaN


In [3]:
from scipy.stats import mode

def get_mode(set):
    if(len(set)): return mode(set, keepdims=False)[0]
    return np.nan

def get_rh(temp, abshum):
    rh = []
    for t, ah in zip(temp, abshum):
        rh.append(ah * (273.15 + t) / (6.112 * np.exp((17.67 * t) / (243.5 + t)) * 2.1674))
    return np.array(rh)

def get_mean(set):
    if len(set): return np.mean(set)
    return np.nan

def is_in_range(value, set) -> bool:
    return set[0] < value < set[-1]

In [4]:
# redo double for
for i in range(np.max(df['Cube ID'].values) + 1):
    cube = df[df['Cube ID'] == i]

    temp_a = np.array(cube['Temperature Layer A'].values)
    rh_a = get_rh(temp_a, np.array(cube['Humidity Layer A'].values))

    temp_b = np.array(cube['Temperature Layer B'].values)
    rh_b = get_rh(temp_b, np.array(cube['Humidity Layer B'].values))

    tmp = np.array(temp_a[~np.isnan(temp_a)])
    umode_ta = get_mode(tmp.round(0))
    dmode_ta = get_mode(tmp[tmp < umode_ta].round(0))

    mean_ha = get_mean(rh_a)

    tmp = np.array(temp_a[~np.isnan(temp_a)])
    umode_tb = get_mode(tmp.round(0))
    dmode_tb = get_mode(tmp[tmp < umode_tb].round(0))

    mean_hb = get_mean(rh_b)
    
    target = []
    for ta, ha, tb, hb in zip(temp_a, rh_a, temp_b, rh_b):
        ta = temp_a[i]
        ha = rh_a[i]

        tb = temp_b[i]
        hb = rh_b[i]

        target_a = False
        target_b = False

        if (is_in_range(ta, [umode_ta - 2, umode_ta + 2]) or \
            is_in_range(ta, [dmode_ta - 2, dmode_ta + 2])) and \
            is_in_range(ha, [mean_ha - 10, mean_ha + 10]):
            target_a = True
        if (is_in_range(tb, [umode_tb - 2, umode_tb + 2]) or \
            is_in_range(tb, [np.array(dmode_tb - 2, umode_tb + 2)])) and \
            is_in_range(hb, [mean_hb - 10, mean_hb + 10]):
            target_b = True
        
        if target_a:
            if target_b: target.append('AB')
            else: target.append('A')
        elif target_b: target.append('B')
        else: target.append('O')

    cube = cube[:] # Copy Warning
    cube['Target'] = target

    df[df['Cube ID'] == i] = cube

In [5]:
df

,Cube ID,Timestamp,Temperature Layer A,Temperature Layer B,Humidity Layer A,Humidity Layer B,Target
0,83,0,23.031810,22.393901,9.915392,8.932601,O
1,87,0,23.251756,23.859351,9.542380,8.506288,O
2,50,8,25.005865,23.974537,9.861271,9.919887,AB
3,40,21,25.315756,24.065631,8.915186,10.003067,O
4,2,21,22.809779,23.765559,10.086120,8.637221,AB
...,...,...,...,...,...,...,...
239209,81,4227968,19.840444,20.517986,8.983224,8.146869,AB
239210,70,4227984,19.914762,20.214499,10.171405,8.755614,AB
239211,34,4227987,19.122549,20.760434,9.814049,7.426978,A
239212,17,4228183,21.117863,21.011024,8.622131,9.234980,AB


In [6]:
df.to_csv('../data/cubes_clustered.csv', index=False)